In [1]:
from hest import iter_hest, VisiumReader
import dask
dask.config.set({'dataframe.query-planning': False})
from pathlib import Path
import hest
print(hest.__version__)
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# check gpu cucim
import cucim
print(cucim.__version__)
from cucim import CuImage

0.0.1


ModuleNotFoundError: No module named 'cucim'

# Slide 1 - 43739

## CAM006

In [4]:
st = VisiumReader().auto_read('/project/simmons_hts/kxu/deepspace_CD/test_data/TIP535JR_FISTULATING/SpaceRanger/')

alignment file is  None


/package/python-cbrg/current/3.11.3/lib/python3.11/site-packages/anndata/_core/anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/package/python-cbrg/current/3.11.3/lib/python3.11/site-packages/anndata/_core/anndata.py:1105: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


AnnData object with n_obs × n_vars = 3928 × 17943
    var: 'gene_ids', 'feature_types', 'genome'
trim the barcodes


/home/k/kxu/.local/lib/python3.11/site-packages/hestcore/wsi.py:27: UserWarning: CuImage is not available. Ensure you have a GPU and cucim installed to use GPU acceleration.
  warnings.warn("CuImage is not available. Ensure you have a GPU and cucim installed to use GPU acceleration.")


In [5]:
st

        'pixel_size' is 0.4401668217373307
        'wsi' is <width=18508, height=20346, backend=NumpyWSI>
        'shapes': []

In [7]:
# ST (adata):
adata = st.adata
print('\n* Scanpy adata:')
print(adata)
adata_hest = adata # backup

# WSI:
wsi = st.wsi
print('\n* WSI:')
print(wsi)

# Shapes:
shapes = st.shapes
print('\n* Shapes:')
print(shapes)

# metadata:
metadata = st.meta
print('\n* Metadata:')
print(metadata)


* Scanpy adata:
AnnData object with n_obs × n_vars = 3928 × 17943
    obs: 'in_tissue', 'array_row', 'array_col', 'pxl_row_in_fullres', 'pxl_col_in_fullres', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: 'spatial'
    obsm: 'spatial'

* WSI:
<width=18508, height=20346, backend=NumpyWSI>

* Shapes:
[]

* Metadata:
{'Sample ID': 'TIP535JR_FISTULATING', 'Number of Spots Under Tissue': 3928, 'Number of Reads': 45702138, 'Mean Reads per Spot': 11634.963849287167, 'Mean Reads Under Tissue per Spot': 10506.661405295315, 'Fraction of Spots Under Tissue': 0.7868589743589743, 'Median Genes per Spot': 1384.5, 'Median UMI Counts per Spot': 2467.5, 'Valid Barcodes': 0.9838933793425594, 'Valid UMI

In [15]:
# Images available
images = spatial_ST.get('images', {})
print("Images available:", list(images.keys()))

# Scale factors
scalefactors = spatial_ST.get('scalefactors', {})
print("Scale factors available:", scalefactors)

Images available: ['downscaled_fullres']
Scale factors available: {'spot_diameter_fullres': 124.95262542259766, 'tissue_downscaled_fullres_scalef': 0.0491497100167109}


In [10]:
def fast_spatial(adata, x, y, spot_size=2, max_cells=10000, color=None, cmap='viridis'):
    """
    Fast spatial plotting using centroid coordinates from Seurat-converted AnnData.
    
    adata: AnnData object
    x, y: obs column names for coordinates
    spot_size: size of points
    max_cells: max number of cells to plot (random subsample for speed)
    color: optional, name of obs column or var (gene) to color by
    cmap: colormap for continuous values
    """
    # Subsample if needed
    if adata.n_obs > max_cells:
        idx = np.random.choice(adata.n_obs, max_cells, replace=False)
        adata_plot = adata[idx, :]
    else:
        adata_plot = adata

    x_vals = adata_plot.obs[x]
    y_vals = adata_plot.obs[y]

    # Handle color
    if color is None:
        c = "grey"
    elif color in adata_plot.obs.columns:
        c = adata_plot.obs[color]
    elif color in adata_plot.var_names:  # gene name
        c = adata_plot[:, color].X.toarray().flatten()
    else:
        raise ValueError(f"{color} not found in .obs or .var")

    plt.figure(figsize=(6, 6))
    sc = plt.scatter(x_vals, y_vals, s=spot_size, c=c, cmap=cmap, alpha=0.7)
    plt.gca().invert_yaxis()  # match spatial orientation
    plt.title(f"Spatial plot ({color if color else 'no color'})")

    if color is not None:
        plt.colorbar(sc, label=color)

    plt.show()
    
fast_spatial(st.adata, x = 'pxl_col_in_fullres', y = 'pxl_row_in_fullres', color = 'total_counts', spot_size=5)

In [13]:
save_dir = Path("/project/simmons_hts/kxu/hest/visium_data")
# # create directory (recursively) if it doesn't exist
save_dir.mkdir(parents=True, exist_ok=True)
st.save_spatial_plot(save_dir)
# Warning saving a large image to pyramidal tiff (>1GB) can be slow on a hard drive.
st.save(save_dir, pyramidal=True)

/home/k/kxu/.local/lib/python3.11/site-packages/hest/HESTData.py:1390: FutureWarning: Use `squidpy.pl.spatial_scatter` instead.
  fig = sc.pl.spatial(adata, show=False, img_key="downscaled_fullres", color=[key], title=f"in_tissue spots", return_fig=True, **pl_kwargs)


saving to pyramidal tiff... can be slow
